# SVM for Coin Classification

This notebook demonstrates the use of a Support Vector Machine (SVM) to classify different coins based on impedance measurements. The data is collected from different groups, and this notebook will preprocess, train, and evaluate the SVM model.


In [1]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from datafilereader import DataFileReader
from datafileviewer_template import DataFileViewer

In [8]:

# Function to load and merge data from all groups for each coin
def load_and_merge_data(groups, coins, data_dir):
    data = []
    labels = []
    for coin in coins:
        coin_name = coin.replace('.h5', '')
        for group in groups:
            file_path = os.path.join(data_dir, group, coin)
            if os.path.isfile(file_path):
                reader = DataFileReader(file_path)
                frequency, Z = reader.get_all_mesurements()
                
                # Separate resistance and reactance
                resistance = np.real(Z)
                reactance = np.imag(Z)
                
                # Extract features (mean and variance of resistance and reactance)
                for idx in range(resistance.shape[0]):
                    features = [
                        np.mean(resistance[idx]), np.var(resistance[idx]),
                        np.mean(reactance[idx]), np.var(reactance[idx])
                    ]
                    data.append(features)
                    labels.append(coin_name)
            # else:
            #     print(f"File not found: {file_path}")
    
    data, labels = shuffle(np.array(data), np.array(labels))
    return pd.DataFrame(data, columns=['res_mean', 'res_var', 'react_mean', 'react_var']), pd.Series(labels)

# Define the directory and groups
data_dir = "./data"
num_groups = 11
groups = [f"Groupe{i}" for i in range(1, num_groups + 1)]
coins = ["1_CHF.h5", "2_CHF.h5", "5_CHF.h5", "10_CTS.h5", "20_CTS.h5", "50_CTS.h5", "EUR_1.h5"]

# Load and merge data
X, y = load_and_merge_data(groups, coins, data_dir)


In [4]:
# Split data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)


In [5]:

# Train the SVM model with a non-linear kernel
svm_model = SVC(kernel='rbf', probability=True)
svm_model.fit(X_train, y_train)

# Evaluate on the validation set
y_val_pred = svm_model.predict(X_val)
print("Validation Set Report")
print(classification_report(y_val, y_val_pred))
print(confusion_matrix(y_val, y_val_pred))


Validation Set Report
              precision    recall  f1-score   support

      10_CTS       0.00      0.00      0.00        24
       1_CHF       0.49      0.86      0.62        29
      20_CTS       0.71      0.59      0.65        34
       2_CHF       0.84      0.90      0.87        29
      50_CTS       0.81      0.97      0.88        35
       5_CHF       0.91      0.89      0.90        46

    accuracy                           0.74       197
   macro avg       0.63      0.70      0.65       197
weighted avg       0.68      0.74      0.70       197

[[ 0 16  4  0  4  0]
 [ 0 25  3  0  1  0]
 [ 0 10 20  0  3  1]
 [ 0  0  1 26  0  2]
 [ 0  0  0  0 34  1]
 [ 0  0  0  5  0 41]]


c:\Users\Zack\miniconda3\envs\ML\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Zack\miniconda3\envs\ML\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Zack\miniconda3\envs\ML\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [6]:
# Evaluate on the test set
y_test_pred = svm_model.predict(X_test)
print("Test Set Report")
print(classification_report(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))


Test Set Report
              precision    recall  f1-score   support

      10_CTS       0.00      0.00      0.00        23
       1_CHF       0.46      0.65      0.54        34
      20_CTS       0.73      0.66      0.69        41
       2_CHF       0.83      0.81      0.82        37
      50_CTS       0.66      0.95      0.78        22
       5_CHF       0.77      0.85      0.81        40

    accuracy                           0.68       197
   macro avg       0.58      0.65      0.61       197
weighted avg       0.62      0.68      0.64       197

[[ 0 14  3  0  6  0]
 [ 0 22  7  0  4  1]
 [ 0 12 27  0  1  1]
 [ 0  0  0 30  0  7]
 [ 0  0  0  0 21  1]
 [ 0  0  0  6  0 34]]


c:\Users\Zack\miniconda3\envs\ML\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Zack\miniconda3\envs\ML\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Zack\miniconda3\envs\ML\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
